In \[28\]:

    import boto3, sys
    import sagemaker
    import pandas as pd
    session   = sagemaker.Session()

In \[ \]:

    #s3://sagemaker-ap-south-1-203504045091/sagemaker/autopilot/input/test/test_data.csv

In \[29\]:

    data = pd.read_csv('test_data.csv')
    data[:10]

Out\[29\]:

|     | No Urgent Place | Friend(s) | Sunny | 80  | 10AM | Carry out & Take away | 2h  | Female | 21  | Unmarried partner | ... | never.1 | Unnamed: 17 | 4\~8 | 1\~3  | 1.1 | 1.2 | 0   | 0.1 | 1.3 | 1.4 |
|-----|-----------------|-----------|-------|-----|------|-----------------------|-----|--------|-----|-------------------|-----|---------|-------------|------|-------|-----|-----|-----|-----|-----|-----|
| 0   | No Urgent Place | Friend(s) | Sunny | 80  | 2PM  | Coffee House          | 2h  | Female | 21  | Unmarried partner | ... | never   | NaN         | 4\~8 | 1\~3  | 1   | 1   | 0   | 0   | 1   | 0   |
| 1   | No Urgent Place | Friend(s) | Sunny | 80  | 6PM  | Restaurant(\<20)      | 2h  | Female | 21  | Unmarried partner | ... | never   | NaN         | 4\~8 | 1\~3  | 1   | 1   | 0   | 0   | 1   | 1   |
| 2   | Work            | Alone     | Sunny | 55  | 7AM  | Bar                   | 1d  | Female | 21  | Unmarried partner | ... | never   | NaN         | 4\~8 | 1\~3  | 1   | 1   | 1   | 0   | 1   | 0   |
| 3   | Work            | Alone     | Sunny | 80  | 7AM  | Restaurant(20-50)     | 1d  | Female | 21  | Unmarried partner | ... | never   | NaN         | 4\~8 | 1\~3  | 1   | 1   | 0   | 0   | 1   | 1   |
| 4   | No Urgent Place | Friend(s) | Sunny | 80  | 2PM  | Coffee House          | 2h  | Male   | 21  | Single            | ... | less1   | 4\~8        | 4\~8 | less1 | 1   | 1   | 0   | 0   | 1   | 0   |
| 5   | No Urgent Place | Friend(s) | Sunny | 80  | 2PM  | Coffee House          | 1d  | Male   | 21  | Single            | ... | less1   | 4\~8        | 4\~8 | less1 | 1   | 1   | 0   | 0   | 1   | 0   |
| 6   | Home            | Alone     | Sunny | 80  | 6PM  | Coffee House          | 2h  | Male   | 21  | Single            | ... | less1   | 4\~8        | 4\~8 | less1 | 1   | 0   | 0   | 0   | 1   | 0   |
| 7   | Work            | Alone     | Sunny | 80  | 7AM  | Restaurant(20-50)     | 1d  | Male   | 21  | Single            | ... | less1   | 4\~8        | 4\~8 | less1 | 1   | 1   | 0   | 0   | 1   | 0   |
| 8   | No Urgent Place | Friend(s) | Sunny | 80  | 10AM | Bar                   | 1d  | Male   | 46  | Single            | ... | 4\~8    | 1\~3        | 1\~3 | never | 1   | 0   | 0   | 0   | 1   | 0   |
| 9   | No Urgent Place | Friend(s) | Sunny | 80  | 2PM  | Coffee House          | 1d  | Male   | 46  | Single            | ... | 4\~8    | 1\~3        | 1\~3 | never | 1   | 0   | 0   | 0   | 1   | 1   |

10 rows × 26 columns

In \[31\]:

    l="No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,1,Some college - no degree,Unemployed,$37500 - $49999,,never,never,,4~8,1~3,1,1,0,0,1"
    end_point_name="coupon-050821"
    response = sagemaker_runtime.invoke_endpoint(EndpointName=end_point_name, ContentType='text/csv', Accept='text/csv', Body=l)
    response = response['Body'].read().decode("utf-8")
    print (response)

    1

In \[36\]:

    tn=tp=fn=fp=count=0

    with open('test_data.csv') as f:
        lines = f.readlines()
        for l in lines[0:10000]:  
            l = l.split(',')      # Split CSV line into features
            label = l[-1]         # Store '1'/'n0' label
            l = l[:-1]            # Remove label
            l = ','.join(l)       # Rebuild CSV line without label
            
            response = sagemaker_runtime.invoke_endpoint(EndpointName=ep_name, 
                                             ContentType='text/csv',       
                                             Accept='text/csv', Body=l)

            response = response['Body'].read().decode("utf-8")
            #print ("label %s response %s" %(label,response))
            if '1' in label:
                # Sample is positive
                if '1' in response:
                    # True positive
                    tp=tp+1
                else:
                    # False negative
                    fn=fn+1
            else:
                # Sample is negative
                if '0' in response:
                    # True negative
                    tn=tn+1
                else:
                    # False positive
                    fp=fp+1
            count = count+1
            if (count % 100 == 0):   
                sys.stdout.write(str(count)+' ')
                
    print ("Done")

    100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 Done

In \[37\]:

    accuracy  = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall    = tp/(tp+fn)
    f1        = (2*precision*recall)/(precision+recall)

    print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

    0.7722 0.7910 0.8203 0.8054